# Projeto 1 - Ciência dos Dados

#### Nome: Andresa Bicudo

#### Nome: Leonardo Malta

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

### TO DO'S:

- Separar emojis (de palavras e de emojis)
- TABELA RELEVANTE ABSOLUTA + 1 / PALAVRAS RELEVANTES + PALAVRAS TOTAIS
- tabela relevante é de todas as palavras, precisamos delas individualmente

### Comentários:
- soma das palavras dos tweets relevantes = palavras_relevantes
- soma das palavras dos tweets irrelevantes = palavras_irrelevantes
- $P(Relevante) =$ palavras_relevantes/palavras_totais
- $P(Irrelevante) =$ palavras_irrelevantes/palavras_totais
- $P(Relevante) + P(Irrelevante) = 1$
- palavras_totais = palavras_relevantes + palavras_irrelevantes
- $P(tweet|Relevante) = $ DADO
- $P(tweet|Irrelevante) = $ DADO
- $P(Relevante|tweet) = $ CLASSIFICADOR
- $P(Irrelevante|tweet) = $ CLASSIFICADOR
- $P(Relevante|tweet) = P(tweet|Relevante)*P(Relevante)/P(tweet)$
- $P(Irrelevante|tweet) = P(tweet|Irrelevante)*P(Irrelevante)/P(tweet)$
- $P(tweet)$ é a probabilidade da frase ocorrer no espaço amostral total (palavras totais) - será desconsiderada porque ambas as probabilidades medidas pelo classificador, que determinarão se o tweet é relevante ou não, serão divididas por ela. Logo, o resultado qualitativo não se alterará, embora o resukltado quantitativo (percentual se alterará)

In [4]:
# Importanto bibliotecas

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import functools
import operator
import re
import emoji

In [5]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\user\Documents\Insper\2° semestre\CDados\Projetos\Projeto1_Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [6]:
dados = pd.read_excel('Corona.xlsx',sheet_name='Treinamento')

In [7]:
dados.tail(5)

,Relevância,Mensagem
495,1,bahia\nclube coloca fazendão à disposição do g...
496,1,rt @clownhejhuff: 4 mortes por corona.... e o ...
497,1,pandemia de corona; cuide-se! #isolamento #is...
498,1,rt @tito_breno: primeiro caso de corona vírus ...
499,1,entrei nos casos com suspeita de corona em tub...


___
## Classificador automático de sentimento


#### Faça aqui uma descrição do seu tema e o que considerou como relevante ou não relevante na classificação dos tweets.

Em meio a essa grande pandemia do COVID-19 (Corona Vírus), é função Ministério da Saúde auxiliar a população com informações de prevenção e de cuidados cajo haja suspeita de contaminação, logo, tweets informativos são relevantes, uma vez que se pode evitar compartilhamento de "fake-news" e manter as recomendações verdadeiras. Ademais, também é essencial que o governo saiba da ocorrência de eventuais eventos que promovem aglomerações e de pessoas que banalizam a situação, a fim de tomar as medidas necessárias para conter o avanço dessa doença. Por fim, como o assunto está sendo extremamente comentado, muitos memes estão sendo feitos, logo, é preciso filtrá-los, recebendo a classificação de não relevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
# Transformando dados em variaveis qualitativas
dados.loc[:,'Mensagem'] = dados['Mensagem'].astype('category')
dados.loc[:,'Relevância'] = dados['Relevância'].astype('category')
dados.Relevância.cat.categories = ['Irrelevante', 'Relevante']
dados.head(5)

,Relevância,Mensagem
0,Relevante,rt @joaopauloz_: saber que algumas pessoas com...
1,Irrelevante,rt @thewayjane: se a campeã não sair daqui o c...
2,Relevante,"na espanha tá feio o corona, tenho medo de que..."
3,Irrelevante,"rt @george_fredson: dos mesmos criadores de ""a..."
4,Irrelevante,as primeiras que vão pega corona vírus aqui na...


In [9]:
# Separando dados em duas tabelas: Relevantes e Irrelevantes 
tweets_relevantes = dados.loc[dados['Relevância'] == 'Relevante', :]
tweets_irrelevantes = dados.loc[dados['Relevância'] == 'Irrelevante', :]

# Teste de separação de mensagem
tweets_irrelevantes.Mensagem[1]

'rt @thewayjane: se a campeã não sair daqui o corona já pode cancelar o bbb #bbb20 https://t.co/z4vvf9qn66'

In [10]:
# Criando listas com os índices dos Tweets relevante e irrelevantes
lista_indices_relevantes = []
lista_indices_irrelevantes = []

for i in range(len(dados.Mensagem)):
    if dados.Relevância[i] == 'Relevante':
        lista_indices_relevantes.append(dados.index[i])
    elif dados.Relevância[i] == 'Irrelevante':
        lista_indices_irrelevantes.append(dados.index[i])

In [11]:
# Função de limpeza que troca alguns sinais, menções e links por espaços, e coloca em uma lista.
# Links de consulta para fazer essa função: 
# https://stackoverflow.com/questions/54733828/remove-twitter-mentions-from-pandas-column
# https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python
# https://stackoverflow.com/questions/12309976/how-do-i-convert-a-list-into-a-string-with-spaces-in-python

def separador_emojis(string):
    string_split_emoji = emoji.get_emoji_regexp().split(string)
    string_split_whitespace = [substr.split() for substr in string_split_emoji]
    string_split = functools.reduce(operator.concat, string_split_whitespace)
    emoji_separados = " ".join(string_split)
    
    return emoji_separados

def cleanup(tweet):
    tweet_sem_mencoes = re.sub("@[A-Za-z0-9_]+","", tweet) # Retirando menções
    tweet_sem_links = re.sub(r'http\S+', '', tweet_sem_mencoes) # Retirando links
    punctuation = '[_ ! - . : ? ; @ , # / " * ( )]' # Sinais que serão substituidos por espaços
    pattern = re.compile(punctuation)
    tweet_sem_pontuacao = re.sub(pattern, ' ', tweet_sem_links) # Retirando sinais
    tweet_emojis_separados = separador_emojis(tweet_sem_pontuacao)
    lista_tweet_semifinal = tweet_emojis_separados.split()
    
    # Removendo itens desnecessários para a classificação (retweets - 'rt', e artigos - 'a','o','as','os')\
    itens_desnecessarios = ['rt','a','o','as','os']  
    for item in itens_desnecessarios:
        for palavra in lista_tweet_semifinal:
            if palavra == item:
                lista_tweet_semifinal.remove(palavra)
                
    lista_tweet_final = lista_tweet_semifinal
    return lista_tweet_final

# Teste do uso da função de limpeza
my_str = cleanup(tweets_irrelevantes.Mensagem[1])
print(my_str)

['se', 'campeã', 'não', 'sair', 'daqui', 'corona', 'já', 'pode', 'cancelar', 'bbb', 'bbb20']


In [12]:
# Colocando todas a palavras relevantes em uma lista, usando a lista de índices criadas anteriormente:
todas_palavras_relevantes = []
todas_palavras_irrelevantes = []

for i in range(len(lista_indices_relevantes)):
    todas_palavras_relevantes.append(cleanup(tweets_relevantes.Mensagem[lista_indices_relevantes[i]]))

for p in range(len(lista_indices_irrelevantes)):
    todas_palavras_irrelevantes.append(cleanup(tweets_irrelevantes.Mensagem[lista_indices_irrelevantes[p]]))

# Trasnformando uma lista de listas em uma única lista:
palavras_relevantes = []
palavras_irrelevantes = []

def Listas2lista(listadelistas):
    nova_lista = []
    for sublista in listadelistas:
        for item in sublista:
            nova_lista.append(item)
    return nova_lista

palavras_relevantes = Listas2lista(todas_palavras_relevantes)
palavras_irrelevantes = Listas2lista(todas_palavras_irrelevantes)

palvras_relevantes_data = pd.Series(palavras_relevantes)
palvras_irrelevantes_data = pd.Series(palavras_irrelevantes)

palavras_totais = palavras_relevantes + palavras_irrelevantes


In [13]:
#Probabilidade de ser Relevante ou Irrelevante dentro de todas as palavras da base de treinamento
P_relevante = len(palavras_relevantes)/(len(palavras_totais))
P_irrelevante = len(palavras_irrelevantes)/(len(palavras_totais))

print('A probabilidade de qualque palavra ser relevante é de {0}%'.format((P_relevante*100)))
print('A probabilidade de qualque palavra ser irrelevante é de {0}%'.format(P_irrelevante*100))

A probabilidade de qualque palavra ser relevante é de 57.10708323396136%
A probabilidade de qualque palavra ser irrelevante é de 42.89291676603864%


In [14]:
# Tabelas de frequências absolutas:
tabela_relevantes_abs = palvras_relevantes_data.value_counts()
tabela_irrelevantes_abs = palvras_irrelevantes_data.value_counts()
tabela_irrelevantes_abs.head()

corona    252
de        102
vírus      87
que        78
eu         71
dtype: int64

In [15]:
# Tabelas de frequências relativas:
tabela_relevantes_rel = palvras_relevantes_data.value_counts(True)
tabela_irrelevantes_rel = palvras_irrelevantes_data.value_counts(True)
tabela_relevantes_rel.head()

corona    0.048236
de        0.036751
que       0.024431
e         0.022552
do        0.021508
dtype: float64

In [16]:
# Essa função calcula a Probabilidade de, dado o tweet, este ser relevante ou irrelevante:

palavras_sem_repetições = list(dict.fromkeys(palavras_totais))

def RetornaFrequencia(palavra, lista_de_palavras):
    frequencia =0
    if palavra in tabela_relevantes_abs:
        frequencia = lista_de_palavras.count(palavra)/len(lista_de_palavras)
    else:
        frequencia = 1/len(lista_de_palavras)
    
    return frequencia

def calcula_probabilidades(tweet, P_relevante, P_irrelevante):
    
    # Uso da função Cleanup para limpar o tweet e trasnformá-lo em uma lista de palavras
    tweet = cleanup(tweet)
    
    P_tweet_dado_relevante = 1
    P_tweet_dado_irrelevante = 1
    
    for i in range(len(tweet)):
        P_tweet_dado_relevante *= RetornaFrequencia(tweet[i], palavras_relevantes)
    
    for i in range(len(tweet)):
        P_tweet_dado_irrelevante *= RetornaFrequencia(tweet[i], palavras_irrelevantes)

    P_relevante_dado_tweet = P_tweet_dado_relevante * P_relevante
    P_irrelevante_dado_tweet = P_tweet_dado_irrelevante * P_irrelevante
    
    probabilidades = [P_irrelevante_dado_tweet, P_relevante_dado_tweet]
    
    return probabilidades
    

In [17]:
# Teste do Classificador com Tweet aleatório
def Classificador(probabilidades):

    if probabilidades[0] > probabilidades[1]:
        return 'Irrelevante'
    else:
        return 'Relevante'
    
tweet_teste = "Para se previnir contra o corona, lave as mãos com água e sabão"

teste = Classificador(calcula_probabilidades(tweet_teste, P_relevante, P_irrelevante))

print(teste)

Relevante


In [18]:
# Teste do Classificador com dados do Treinamento
resultado = []
for i in range(len(dados.Mensagem)):
    resultado.append(Classificador(calcula_probabilidades(dados.Mensagem[i], P_relevante, P_irrelevante)))

dados['Classificação Treinamento'] = resultado
dados.head()

,Relevância,Mensagem,Classificação Treinamento
0,Relevante,rt @joaopauloz_: saber que algumas pessoas com...,Relevante
1,Irrelevante,rt @thewayjane: se a campeã não sair daqui o c...,Irrelevante
2,Relevante,"na espanha tá feio o corona, tenho medo de que...",Relevante
3,Irrelevante,"rt @george_fredson: dos mesmos criadores de ""a...",Irrelevante
4,Irrelevante,as primeiras que vão pega corona vírus aqui na...,Irrelevante


In [19]:
x = dados['Classificação Treinamento'][1]
print(x)

Irrelevante


In [20]:
acertou = 0
errou = 0

for i in range(0,len(dados['Relevância'])):
    if dados['Relevância'][i] == dados['Classificação Treinamento'][i]:
        acertou += 1
    else:
        errou += 1
    
print(acertou)
print(errou)

473
27


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [21]:
# Baixando Excel sheet "Teste"
dados_teste = pd.read_excel('Corona.xlsx',sheet_name='Teste')
dados_teste.tail()

,Relevância,Mensagem
363,1.0,índice de medo provocado pelo corona vírus pro...
364,1.0,nessa cena vemos algumas imbecilidades:\n\n1- ...
365,1.0,"não aguento esses artistas fazendo drama, enqu..."
366,1.0,rt @andreluiz332: o @lulaoficial viajou a eur...
367,1.0,"mano, o brasileiro médio precisa estar morrend..."


In [22]:
# Transformando dados em variaveis qualitativas
dados_teste.loc[:,'Mensagem'] = dados['Mensagem'].astype('category')
dados_teste.loc[:,'Relevância'] = dados['Relevância'].astype('category')
dados_teste.Relevância.cat.categories = ['Irrelevante', 'Relevante']
dados_teste.head(5)

,Relevância,Mensagem
0,Relevante,rt @joaopauloz_: saber que algumas pessoas com...
1,Irrelevante,rt @thewayjane: se a campeã não sair daqui o c...
2,Relevante,"na espanha tá feio o corona, tenho medo de que..."
3,Irrelevante,"rt @george_fredson: dos mesmos criadores de ""a..."
4,Irrelevante,as primeiras que vão pega corona vírus aqui na...


In [23]:
# Separando dados em duas tabelas: Relevantes e Irrelevantes 
tweets_relevantes_teste = dados.loc[dados['Relevância'] == 'Relevante', :]
tweets_irrelevantes_teste = dados.loc[dados['Relevância'] == 'Irrelevante', :]

# Teste de separação de mensagem
tweets_irrelevantes_teste.Mensagem[1]

'rt @thewayjane: se a campeã não sair daqui o corona já pode cancelar o bbb #bbb20 https://t.co/z4vvf9qn66'

In [24]:
# Criando listas com os índices dos Tweets relevante e irrelevantes
lista_indices_relevantes_teste = []
lista_indices_irrelevantes_teste = []

for i in range(len(dados_teste.Mensagem)):
    if dados_teste.Relevância[i] == 'Relevante':
        lista_indices_relevantes_teste.append(dados.index[i])
    elif dados_teste.Relevância[i] == 'Irrelevante':
        lista_indices_irrelevantes_teste.append(dados.index[i])

In [25]:
# Teste do uso da função de limpeza
my_str = cleanup(tweets_irrelevantes_teste.Mensagem[1])
print(my_str)

['se', 'campeã', 'não', 'sair', 'daqui', 'corona', 'já', 'pode', 'cancelar', 'bbb', 'bbb20']


In [26]:
# Para o teste, utilizaremos os dados do treinamento
resultado_teste = []
for i in range(len(dados_teste.Mensagem)):
    resultado_teste.append(Classificador(calcula_probabilidades(dados_teste.Mensagem[i], P_relevante, P_irrelevante)))

dados_teste['Classificação Treinamento'] = resultado_teste
dados_teste.tail()

,Relevância,Mensagem,Classificação Treinamento
363,Irrelevante,"rt @brunofe53103298: corona virus que lute, ma...",Irrelevante
364,Relevante,rt @carolinatmoura: única preocupação em relaç...,Relevante
365,Irrelevante,rt @habbid: cof cof cof... calma gente não é c...,Irrelevante
366,Relevante,rt @apfaqui: quer tanto que achem a cura do co...,Relevante
367,Irrelevante,rt @dougllasssd: corona vírus não atingirá nem...,Irrelevante


In [27]:
# Contabilizador de erros e acertos
acertou_teste = 0
errou_teste = 0

for i in range(0,len(dados_teste['Relevância'])):
    if dados_teste['Relevância'][i] == dados_teste['Classificação Treinamento'][i]:
        acertou_teste += 1
    else:
        errou_teste += 1
    
print(acertou_teste)
print(errou_teste)

349
19


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [28]:
# https://stackoverflow.com/questions/54733828/remove-twitter-mentions-from-pandas-column
# https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python
# https://stackoverflow.com/questions/12309976/how-do-i-convert-a-list-into-a-string-with-spaces-in-python